In [46]:
import pandas as pd

In [76]:
df = pd.read_csv('atocha.csv', index_col = 0)
df.shape

(2475, 42)

In [77]:
df = df[['price', 'propertyType', 'size', 'exterior', 'rooms',
         'bathrooms', 'address', 'province', 'municipality',
         'latitude', 'longitude', 'status', 'newDevelopment', 'hasLift',
         'parkingSpace', 'priceByArea', 'detailedType', 'highlight']]

In [78]:
df

,price,propertyType,size,exterior,rooms,bathrooms,address,province,municipality,latitude,longitude,status,newDevelopment,hasLift,parkingSpace,priceByArea,detailedType,highlight
0,310000.0,flat,72.0,True,3,1,Calle de Canarias,Madrid,Madrid,40.402401,-3.689032,good,False,False,NaN,4306.0,{'typology': 'flat'},{'groupDescription': 'Destacado'}
1,228000.0,flat,42.0,False,2,1,Calle de Canarias,Madrid,Madrid,40.403476,-3.687239,good,False,False,NaN,5429.0,{'typology': 'flat'},{'groupDescription': 'Destacado'}
2,159700.0,flat,51.0,True,1,1,Calle del General Lacy,Madrid,Madrid,40.403561,-3.689225,renew,False,True,NaN,3131.0,{'typology': 'flat'},{'groupDescription': 'Destacado'}
3,230000.0,flat,47.0,False,2,1,barrio Palos de Moguer,Madrid,Madrid,40.402347,-3.690002,good,False,True,NaN,4894.0,{'typology': 'flat'},NaN
4,170000.0,flat,27.0,False,1,1,Calle Ancora,Madrid,Madrid,40.400857,-3.688740,good,False,False,NaN,6296.0,{'typology': 'flat'},NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2470,899000.0,flat,88.0,True,2,2,barrio Sol,Madrid,Madrid,40.418637,-3.702219,good,False,True,NaN,10216.0,{'typology': 'flat'},NaN
2471,1400000.0,flat,160.0,True,3,2,barrio Palacio,Madrid,Madrid,40.415068,-3.707434,good,False,False,NaN,8750.0,{'typology': 'flat'},NaN
2472,380000.0,flat,70.0,True,3,1,Plaza de la Paja,Madrid,Madrid,40.411305,-3.710698,good,False,True,NaN,5429.0,{'typology': 'flat'},{'groupDescription': 'Top'}
2473,155000.0,flat,52.0,True,2,1,sancho panza,Madrid,Madrid,40.389461,-3.669344,good,False,False,NaN,2981.0,{'typology': 'flat'},{'groupDescription': 'Destacado'}


In [79]:
# Exterior cleaned
df['exterior'] = df['exterior'].fillna(False)

In [80]:
# hasLift cleaned
df['hasLift'] = df['hasLift'].fillna(False)

In [81]:
# highlight cleaned
df['highlight'] = df['highlight'].fillna("{'groupDescription': None}")

In [82]:
# parkingSpace cleaned
df['parkingSpace'] = df['parkingSpace'].fillna("{'hasParkingSpace': False, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': None}")

# Feature engineering

In [83]:
# parkingSpace included fixed
df['parkingSpace'] = df['parkingSpace'].apply(lambda x: "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': True, 'parkingSpacePrice': None}" if x == "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': True}" else x)

In [84]:
# unique parkingSpaces are now consistent
df['parkingSpace'].unique()

array(["{'hasParkingSpace': False, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': None}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': True, 'parkingSpacePrice': None}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 30000.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 25000.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 40000.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 20000.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 35000.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': True, 'parkingSpacePrice': 15000.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 9400.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceInclu

In [85]:
# fixing detailedType
df['detailedType'] = df['detailedType'].apply(lambda x: "{'typology': 'flat', 'subTypology': 'standardFlat'}" if x == "{'typology': 'flat'}" else x)
df['detailedType'] = df['detailedType'].apply(lambda x: "{'typology': 'chalet', 'subTypology': 'standardChalet'}" if x == "{'typology': 'chalet'}" else x)

In [86]:
# Converting string of dicts --> dicts

from ast import literal_eval # Library for string evaluation

df['parkingSpace'] = df['parkingSpace'].apply(literal_eval)
df['highlight'] = df['highlight'].apply(literal_eval)
df['detailedType'] = df['detailedType'].apply(literal_eval)

In [87]:
df = pd.concat([df,
           pd.json_normalize(df['parkingSpace']),
           pd.json_normalize(df['highlight']),
           pd.json_normalize(df['detailedType'])],
          axis = 1)

In [88]:
df

,price,propertyType,size,exterior,rooms,bathrooms,address,province,municipality,latitude,...,parkingSpace,priceByArea,detailedType,highlight,hasParkingSpace,isParkingSpaceIncludedInPrice,parkingSpacePrice,groupDescription,typology,subTypology
0,310000.0,flat,72.0,True,3,1,Calle de Canarias,Madrid,Madrid,40.402401,...,"{'hasParkingSpace': False, 'isParkingSpaceIncl...",4306.0,"{'typology': 'flat', 'subTypology': 'standardF...",{'groupDescription': 'Destacado'},False,False,NaN,Destacado,flat,standardFlat
1,228000.0,flat,42.0,False,2,1,Calle de Canarias,Madrid,Madrid,40.403476,...,"{'hasParkingSpace': False, 'isParkingSpaceIncl...",5429.0,"{'typology': 'flat', 'subTypology': 'standardF...",{'groupDescription': 'Destacado'},False,False,NaN,Destacado,flat,standardFlat
2,159700.0,flat,51.0,True,1,1,Calle del General Lacy,Madrid,Madrid,40.403561,...,"{'hasParkingSpace': False, 'isParkingSpaceIncl...",3131.0,"{'typology': 'flat', 'subTypology': 'standardF...",{'groupDescription': 'Destacado'},False,False,NaN,Destacado,flat,standardFlat
3,230000.0,flat,47.0,False,2,1,barrio Palos de Moguer,Madrid,Madrid,40.402347,...,"{'hasParkingSpace': False, 'isParkingSpaceIncl...",4894.0,"{'typology': 'flat', 'subTypology': 'standardF...",{'groupDescription': None},False,False,NaN,None,flat,standardFlat
4,170000.0,flat,27.0,False,1,1,Calle Ancora,Madrid,Madrid,40.400857,...,"{'hasParkingSpace': False, 'isParkingSpaceIncl...",6296.0,"{'typology': 'flat', 'subTypology': 'standardF...",{'groupDescription': None},False,False,NaN,None,flat,standardFlat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2470,899000.0,flat,88.0,True,2,2,barrio Sol,Madrid,Madrid,40.418637,...,"{'hasParkingSpace': False, 'isParkingSpaceIncl...",10216.0,"{'typology': 'flat', 'subTypology': 'standardF...",{'groupDescription': None},False,False,NaN,None,flat,standardFlat
2471,1400000.0,flat,160.0,True,3,2,barrio Palacio,Madrid,Madrid,40.415068,...,"{'hasParkingSpace': False, 'isParkingSpaceIncl...",8750.0,"{'typology': 'flat', 'subTypology': 'standardF...",{'groupDescription': None},False,False,NaN,None,flat,standardFlat
2472,380000.0,flat,70.0,True,3,1,Plaza de la Paja,Madrid,Madrid,40.411305,...,"{'hasParkingSpace': False, 'isParkingSpaceIncl...",5429.0,"{'typology': 'flat', 'subTypology': 'standardF...",{'groupDescription': 'Top'},False,False,NaN,Top,flat,standardFlat
2473,155000.0,flat,52.0,True,2,1,sancho panza,Madrid,Madrid,40.389461,...,"{'hasParkingSpace': False, 'isParkingSpaceIncl...",2981.0,"{'typology': 'flat', 'subTypology': 'standardF...",{'groupDescription': 'Destacado'},False,False,NaN,Destacado,flat,standardFlat


In [89]:
del df['parkingSpace']
del df['highlight']
del df['detailedType']

In [90]:
df2 = df

In [91]:
df = pd.read_csv('recleaned_df.csv', index_col = 0)

In [92]:
df.columns == df2.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [93]:
combined = pd.concat([df, df2], axis = 0)
combined.shape

(7826, 21)

In [94]:
combined.drop_duplicates()

,price,propertyType,size,exterior,rooms,bathrooms,address,province,municipality,latitude,...,status,newDevelopment,hasLift,priceByArea,hasParkingSpace,isParkingSpaceIncludedInPrice,parkingSpacePrice,groupDescription,typology,subTypology
0,335000.0,flat,136.0,True,3,2,SIGRID,Madrid,Rivas-Vaciamadrid,40.352160,...,good,False,True,2463.0,True,True,NaN,Destacado,flat,standardFlat
1,1650000.0,flat,179.0,True,2,3,Calle del Príncipe de Vergara,Madrid,Madrid,40.433593,...,good,False,True,9218.0,True,True,NaN,Destacado,flat,standardFlat
2,770000.0,flat,109.0,True,2,2,barrio Ibiza,Madrid,Madrid,40.416551,...,good,False,True,7064.0,False,False,NaN,Destacado,flat,standardFlat
3,175000.0,flat,49.0,True,1,1,"Calle de los Morales, 10",Madrid,Madrid,40.361994,...,good,False,True,3571.0,True,False,25000.0,Destacado,flat,standardFlat
4,280000.0,flat,121.0,True,4,2,Calle del Río Ulla,Madrid,Madrid,40.432982,...,good,False,False,2314.0,False,False,NaN,Destacado,flat,standardFlat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2470,899000.0,flat,88.0,True,2,2,barrio Sol,Madrid,Madrid,40.418637,...,good,False,True,10216.0,False,False,NaN,None,flat,standardFlat
2471,1400000.0,flat,160.0,True,3,2,barrio Palacio,Madrid,Madrid,40.415068,...,good,False,False,8750.0,False,False,NaN,None,flat,standardFlat
2472,380000.0,flat,70.0,True,3,1,Plaza de la Paja,Madrid,Madrid,40.411305,...,good,False,True,5429.0,False,False,NaN,Top,flat,standardFlat
2473,155000.0,flat,52.0,True,2,1,sancho panza,Madrid,Madrid,40.389461,...,good,False,False,2981.0,False,False,NaN,Destacado,flat,standardFlat


In [95]:
combined.to_csv('recleaned_df.csv')